<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Modelisation_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [2]:
from google.colab import files
uploaded = files.upload()


import pandas as pd
from PIL import Image
from torchvision import transforms
import numpy as np

In [3]:
train_df_images = pd.read_json('train_df_images.json', orient="records", lines=True)

train_df_images

,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1..."
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2..."
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,"[[[109, 110, 103], [113, 115, 107], [117, 118,..."
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]..."
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,"[[[106, 88, 76], [114, 96, 84], [115, 96, 84],..."
...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,None
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,None
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,"[[[61, 117, 223], [64, 118, 226], [65, 119, 22..."
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,"[[[153, 147, 44], [209, 198, 54], [201, 192, 5..."


In [3]:
#train_df_images['image'] = train_df_images['image'].apply(lambda x: np.array(x))

Mapping labels to integers

In [4]:
labels = train_df_images['label'].unique()
dict_mapping ={}

for i, label in enumerate(labels):
  dict_mapping[label]=int(i)
  i = i+1

print(dict_mapping)

train_df_images['label'] = train_df_images['label'].map(dict_mapping)
train_df_images['label'] = train_df_images['label'].apply(lambda x: np.array(x))

{'cultural exclusive': 0, 'cultural representative': 1, 'cultural agnostic': 2, 'cultural': 3, 'cult': 4, None: 5, 'cultural agn': 6, 'cultural represent': 7, 'cultural ex': 8, 'cultural ag': 9}


In [5]:
train_df_images.head()

,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,0,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1..."
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,1,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2..."
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,1,"[[[109, 110, 103], [113, 115, 107], [117, 118,..."
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,1,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]..."
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,1,"[[[106, 88, 76], [114, 96, 84], [115, 96, 84],..."


# Missing values

Some photos are missing for many items, and as the CNN can not handles missing values, we have to find solutions to deal with this problem. We have for now 2 options :

In [6]:
def fill_NaN_image(image):

  """
  If a cell is empty, it fills in with a white image, of dimension (28,28,3) ie (height, width, channels)
  """

  if image is None or (isinstance(image, float) and np.isnan(image)):
    image = np.ones((28, 28, 3), dtype=np.uint8) * 255

  return image

# Test
#image = train_df_images['train_df_images'][6247]
#print(fill_NaN_image(image))

In [7]:
train_df_images['image'] = train_df_images['image'].apply(lambda x: fill_NaN_image(x))

In [8]:
train_df_images['image'] = train_df_images['image'].apply(lambda x: np.array(x))

Data augmentation, as we don't have a lot of data

In [ ]:
#transform = transforms.Compose([
#    transforms.RandomResizedCrop(size=28, scale=(0.8, 1.0)),  # zoom entre 80% et 100%
#    transforms.RandomHorizontalFlip(p=0.5),
#    transforms.RandomRotation(degrees=15),
#    transforms.ToTensor()
#])

#nb_augmentation = 3

#for i in range(3):
#  train_df_images[f'augmented_image_{i}'] = None
#  train_df_images[f'augmented_image_{i}'] = train_df_images['image'].apply(lambda x: transform(Image.fromarray(x.astype(np.uint8))) if isinstance(x, np.ndarray) else None)

#train_df_images.head()

Modelisation

In [ ]:
#for i in range(len(train_df_images)):
#  if train_df_images['image'][i].shape != (28, 28, 3):
#    print(train_df_images['image'][i].shape)

In [13]:
# Normalisation so that the fitting of the CNN is easier
X_train = np.stack(train_df_images['image'].to_numpy()).astype('float32') / 255.0
y_train = train_df_images['label']

# Checkings
print(X_train.shape)
print(X_train[0].shape)
print(type(X_train[0]))

print(y_train.shape)
print(y_train[0])
print(np.unique(y_train))

(6251, 28, 28, 3)
(28, 28, 3)
<class 'numpy.ndarray'>
(6251,)
0
[0 1 2 3 4 5 6 7 8 9]


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

model = models.Sequential([
    #Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)),
    layers.MaxPooling2D((2, 2)),

    # Layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Layer 3
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),

    # If we want a classification. Otherwise, we can just give as output, the channels, wich will be used in another model.
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax') # CAUTION, we have to change the number of layer at the end, if we want only 3 labels. Here the data is not clean, so we still have 10 labels
])

# Compilation
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training
model.fit(X_train, y_train, epochs= 40, batch_size=32)

Epoch 1/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.4245 - loss: 1.1748
Epoch 2/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.4424 - loss: 1.0437
Epoch 3/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.4474 - loss: 1.0424
Epoch 4/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.4444 - loss: 1.0388
Epoch 5/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.4704 - loss: 1.0324
Epoch 6/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.4885 - loss: 0.9952
Epoch 7/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.4798 - loss: 1.0030
Epoch 8/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.4885 - loss: 0.9921
Epoch 9/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.5085 - loss: 0.9727
Epoch 10/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.5081 - loss: 0.9783
Epoch 11/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.5172 - loss: 0.9716
Epoch 12/40
196/196 ━━━━━━━━━━━━━━━━━